# Eradiate — Level-1 and Level-2 Interfaces

In this tutorial, we will see how to interact with Eradiate using its less abstract interfaces. Throughout our progression, we will progressively build a simple scene, visualise it and perform very simple computations. We will explore the many possiblities we have to generate our scene and see in which situation each interface is best.

## 1. The kernel interface (Level-1)

We will first see how to directly interact with Eradiate's computational kernel. Loading the kernel submodule is as simple as importing it:

In [ ]:
import eradiate.kernel

This will load the embedded Mitsuba kernel. Prior to usage, selecting a Mitsuba variant is mandatory. This is done using the `set_variant` function. We are going to work in monochromatic mode, without polarisation, and we therefore select the double-precision monochromatic variant:

In [ ]:
eradiate.kernel.set_variant("scalar_mono_double")

Once this is done, we can load a scene specified as an XML file. The proposed scene consists of a rectangular surface topped by a slab of participating medium featuring only Rayleigh scattering. The scene is illuminated by a directional source with a intensity of the same order of magnitude as the Sun's. We can take a look at the XML file, then come back here.

To load the scene into the computational kernel, we use the `load_file` function:

In [ ]:
from eradiate.kernel.core.xml import load_file
scene = load_file("scene.xml")

Note that the scene file has variable arguments which can be used to specify scene parameters upon loading. For example, we can change the resolution of the perspective camera as well as the number of samples per pixel:

In [ ]:
scene = load_file("scene.xml", res=64, spp=512)

Since the `Scene` class has a string representation, we can visualise the scene contents:

In [ ]:
scene

Running the computation defined in the scene file is then done by calling the integrator's `render` method:

In [ ]:
sensor = scene.sensors()[0]
scene.integrator().render(scene, sensor)

For future use, we'll pack this into a function.

In [ ]:
def render(scene):
    sensor = scene.sensors()[0]
    scene.integrator().render(scene, sensor)

Once the computation is complete, we can visualise the results using the matplotlib plotting package:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.array(sensor.film().bitmap())
plt.imshow(np.squeeze(data))
plt.colorbar()
plt.show()
plt.close()

Again, we'll write this as a function for convenience:

In [ ]:
def plot(scene):
    import numpy as np
    import matplotlib.pyplot as plt

    sensor = scene.sensors()[0]
    data = np.array(sensor.film().bitmap())
    plt.imshow(np.squeeze(data))
    plt.colorbar()
    plt.show()
    plt.close()

We now change the scene parameters to increase the number of samples per pixel so as to reduce noise:

In [ ]:
scene = load_file("scene.xml", res=64, spp=4096)
render(scene)
plot(scene)

The scene we just rendered can also be loaded from a Python dictionary. It is very important, however, to note that building a scene dictionary requires to select a kernel variant.

In [ ]:
import eradiate.kernel
eradiate.kernel.set_variant("scalar_mono_double")
from eradiate.kernel.core import ScalarTransform4f

scene_dict = {
    "type": "scene",
    
    # Flat rectangular diffuse surface with reflectance of 0.5
    # and dimensions 200 km x 200 km
    "surface": {
        "type": "rectangle",
        "to_world": ScalarTransform4f.scale([100, 100, 1]),
        "bsdf": {
            "type": "diffuse",
            "reflectance": {"type": "uniform", "value": 0.5}
        }
    },

    # Homogeneous rayleigh-scattering plane-parallel atmosphere 
    # with dimensions 200 km x 200 km x 40 km. 
    # Value of sigma_t is arbitrary but realistic.
    "atmosphere": {
        "type": "cube",
        "to_world": 
        ScalarTransform4f.translate([0, 0, 20.001]) *
        ScalarTransform4f.scale([100, 100, 20])
        ,
        # The shape is translated upward so that the bottom face does
        # not overlap with the surface
        "bsdf": {"type": "null"},
        "interior": {
            "type": "homogeneous",
            "phase": {"type": "rayleigh"},
            "sigma_t": {
                "type": "uniform",
                "value": 1.0e-2
            },
            "albedo": {
                "type": "uniform",
                "value": 1.0
            }
        }
    },

    # Directional light source with zenith angle of 30° and 
    # irradiance of 1.8 W/km^2/nm.
    "illumination": {
        "type": "directional",
        "direction": [-0.5, 0, -0.8660254],
        "irradiance": {
            "type": "uniform",
            "value": 1.8e6
        }
    },

    # Perspective camera with a 45° zenith viewing angle
    "measure": {
        "type": "perspective",
        "sampler": {
            "type": "independent",
            "sample_count": 4096
        },
        "film": {
            "type": "hdrfilm",
            "width": 64,
            "height": 64,
            "component_format": "float32",
            "pixel_format": "luminance",
            "rfilter": {"type": "box"},
        },
        "far_clip": 1e7,
        "to_world": ScalarTransform4f
            .look_at(origin=[0, 400, 400],
                     target=[0, 0, 0],
                     up=[0, 0, 1])
    },

    # Volumetric path tracer (no multiple importance sampling)
    "integrator": {"type": "volpath"}
}
display(scene_dict)

from eradiate.kernel.core.xml import load_dict
scene = load_dict(scene_dict)

The scene can then be rendered using the facilities we created before:

In [ ]:
render(scene)
plot(scene)

Kernel scene dictionaries are a very flexible way of interacting with the model. They are very easy to modify. For instance, let's add a few objects to the scene (floating spheres), increase the image resolution and decrease the number of samples:

In [ ]:
scene_dict["measure"]["sampler"]["sample_count"] = 512
scene_dict["measure"]["film"]["height"] = scene_dict["measure"]["film"]["width"] = 256
scene_dict["integrator"]["type"] = "volpath"

n_spheres = 24
angles = np.linspace(0, 2. * np.pi, n_spheres, endpoint=False)
cycle = 6

reflectances_fwd = np.linspace(0., 1., int(cycle/2)+1)
reflectances_bwd = reflectances_fwd[-2:-int(cycle/2)-1:-1]
reflectances = (list(reflectances_fwd) + list(reflectances_bwd)) * int(n_spheres/cycle)
 
for i, (angle, reflectance) in enumerate(zip(angles, reflectances)):
    scene_dict[f"floating_sphere_{i}"] = {
        "type": "sphere", 
        "radius": 7.5,
        "center": [75. * np.cos(angle), 75. * np.sin(angle), 50],
        "bsdf": {
            "type": "diffuse",
            "reflectance": {
                "type": "uniform",
                "value": reflectance * 0.75
            }
        }
    }
scene = load_dict(scene_dict)
render(scene)
plt.figure(figsize=(8,6))
plot(scene)

# 2. The scene generation interface (Level-2)

As we can see, building a scene can become a tedious task. As scenes grow in size and complexity, writing XML files or Python dictionaries manually will become impractical—if not impossible. We typically would like to have a tool to generate kernel scenes from a convenient set of instructions.

The `eradiate.scenes` package serves this purpose and contains a set of convenience classes and functions to assist the user with scene creation. Let's see how we can generate the scene we've been working on so far.

The `SceneDict` class wraps an ordinary scene dictionary. It keeps track of the kernel variant with which it has been created to avoid inconsistencies: a scene dictionary created for a given kernel variant cannot be used with another kernel variant and `SceneDict` will report clearly this sort of inconsistencies.

In [ ]:
import eradiate.kernel
from eradiate.scenes import SceneDict
eradiate.kernel.set_variant("scalar_mono_double")


scene_dict = SceneDict.empty()
print(f"scene_dict.variant = {scene_dict.variant}")
display(scene_dict)

Now, we can start building our scene. We first add a Lambertian surface using the `Lambertian` class. A `Lambertian` instance is initialised using a dictionary which contains a set of parameters specified in the class's documentation.

In [ ]:
from eradiate.scenes.lithosphere import Lambertian

scene_horizontal_size = 200.  # km

surface = Lambertian({
    "reflectance": 0.5, 
    "width": scene_horizontal_size,
})
display(surface)

The `Lambertian` instance can be converted into a dictionary suitable for merge with a kernel scene. By default, the BSDF plugin is defined at the top level of the scene, which allows to share it between multiple shapes and minimise resource cost.

In [ ]:
d = surface.kernel_dict()
display(d)

Each item of the dictionary can be instantiated separately:

In [ ]:
from eradiate.kernel.core.xml import load_dict
print(load_dict(surface.kernel_dict()["bsdf_surface"]))

We can add it to our scene:

In [ ]:
scene_dict.add(surface)
display(scene_dict)

Now, we can move on to adding the participating medium. This is done using the `RayleighHomogeneous` class.

In [ ]:
from eradiate.scenes.atmosphere import RayleighHomogeneous

atmosphere = RayleighHomogeneous({
    "sigma_s": 1e-2,  # km^-1
    "height": 40.,  # km
    "width": scene_horizontal_size  # km
})

scene_dict.add(atmosphere)

We now need to illuminate our scene. For that purpose, we'll use the ``Directional`` class. It is parametrised in a natural way for EO scientists, using a (zenith, azimuth) pair.

In [ ]:
from eradiate.scenes.illumination import Directional

illumination = Directional({
    "zenith": 30.,
    "azimuth": 0.,
    "irradiance": 1.8e+6  # W/km^2/nm
})

scene_dict.add(illumination)

One last thing is missing in our scene: we need a sensor to record light. Since we are rebuilding the scene we created before, we'll add a perspective camera:

In [ ]:
from eradiate.scenes.measure import Perspective

scene_dict.add(Perspective({
    "target": [0, 0, 0],
    "zenith": 45.,
    "azimuth": 90.,
    "distance": 400. * np.sqrt(2),
    "res": 64,
    "spp": 4096,
}))

Our scene is now only missing an integrator, which will specify the integration algorithm. Since we have a participating medium, we'll need a volume-enabled algorithm and use the volumetric path tracer plugin. We can mix Eradiate's scene generation helpers with kernel scene specification dictionaries:

In [ ]:
scene_dict.add({"integrator": {"type": "volpath"}})

We are now ready to run the simulation:

In [ ]:
scene = scene_dict.load()
render(scene)
plot(scene)

For comparison, this is what the complete scene creation code looks like:

In [ ]:
scene_dict = SceneDict.empty().add([
    Lambertian({
        "reflectance": 0.5, 
        "width": 200
    }),
    RayleighHomogeneous({
        "sigma_s": 1e-2,
        "height": 40.,
        "width": 200.
    }),
    Directional({
        "zenith": 30.,
        "azimuth": 0.,
        "irradiance": 1.8e+6
    }),
    Perspective({
        "target": [0, 0, 0],
        "zenith": 45.,
        "azimuth": 90.,
        "distance": 400. * np.sqrt(2),
        "res": 64,
        "spp": 4096,
    }),
    {"integrator": {"type": "volpath"}}
])

For comparison, the same scene created using the Level-1 interface looks like this:

In [ ]:
scene_dict_manual = {
    "type": "scene",
    
    # Flat rectangular diffuse surface with reflectance of 0.5
    # and dimensions 200 km x 200 km
    "surface": {
        "type": "rectangle",
        "to_world": ScalarTransform4f.scale([100, 100, 1]),
        "bsdf": {
            "type": "diffuse",
            "reflectance": {"type": "uniform", "value": 0.5}
        }
    },

    # Homogeneous rayleigh-scattering plane-parallel atmosphere 
    # with dimensions 200 km x 200 km x 40 km. 
    # Value of sigma_t is arbitrary.
    "atmosphere": {
        "type": "cube",
        "to_world": 
            ScalarTransform4f.translate([0, 0, 20.001]) *
            ScalarTransform4f.scale([100, 100, 20]),
        # The shape is translated upward so that the bottom face does
        # not overlap with the surface
        "bsdf": {"type": "null"},
        "interior": {
            "type": "homogeneous",
            "phase": {"type": "rayleigh"},
            "sigma_t": {
                "type": "uniform",
                "value": 1.0e-2
            },
            "albedo": {
                "type": "uniform",
                "value": 1.0
            }
        }
    },

    # Directional light source with zenith angle of 30° and 
    # irradiance of 1 W/km^2/nm.
    "illumination": {
        "type": "directional",
        "direction": [-0.5, 0, -0.8660254],
        "irradiance": {
            "type": "uniform",
            "value": 1e6
        }
    },

    # Perspective camera with a 45° zenith viewing angle
    "measure": {
        "type": "perspective",
        "sampler": {
            "type": "independent",
            "sample_count": 4096
        },
        "film": {
            "type": "hdrfilm",
            "width": 64,
            "height": 64,
            "component_format": "float32",
            "pixel_format": "luminance",
            "rfilter": {"type": "box"},
        },
        "far_clip": 1e7,
        "to_world": ScalarTransform4f
            .look_at(origin=[0, 400, 400],
                     target=[0, 0, 0],
                     up=[0, 0, 1])
    },

    # Volumetric path tracer (no multiple importance sampling)
    "integrator": {"type": "volpath"}
}

# 3. One-dimensional solver interface

Eradiate ships classes useful to perform specific computations. In this example, we will see how to use a generic one-dimensional solver used as the basic core of more complex applications.

We start by importing the kernel and selecting a variant. We will run monochromatic simulations and therefore select the `scalar_mono_double` variant.

In [ ]:
import eradiate.kernel
eradiate.kernel.set_variant("scalar_mono_double")

We now instantiate the `OneDimSolver` class. It takes a scene geometry, optical properties and illumination, and positions sensors automatically so as to compute the radiance leaving the scene at the top of the atmosphere.

`OneDimSolver` is designed to host pseudo one-dimensional scene, _i.e._ solve problems with two translational invariances. However, Eradiate's Mitsuba kernel does not exactly handle this type of symmetries; instead, a normal three-dimensional scene is used. This means that special care must be taken when creating the scene. The scene we have been working on in the rest of this tutorial is designed with that in mind: its optical thickness in the horizontal directions is such that the point (0,0,0) is out of the radiative boundary layer and will therefore not be subject to boundary effects.

In [ ]:
from eradiate.solvers.onedim import OneDimSolver
from eradiate.scenes import SceneDict
from eradiate.scenes.lithosphere import Lambertian
from eradiate.scenes.atmosphere import RayleighHomogeneous
from eradiate.scenes.illumination import Directional

solver = OneDimSolver(SceneDict.empty().add([
    Lambertian({
        "type": "lambertian",
        "reflectance": 0.5, 
        "width": 200
    }),
    RayleighHomogeneous({
        "sigma_s": 1e-2,
        "height": 40.,
        "width": 200.
    }),
    Directional({
        "zenith": 0.,
        "azimuth": 0.,
        "irradiance": 1.0e+6
    }),
    {"integrator": {"type": "volpathmis"}}
]))

Note that this scene definition does _not_ include any sensor: the solver will take care of instantiating them and positioning them appropriately based on the angular configuration requested by the user.

We can first run the simulation for a single angular configuration:

In [ ]:
solver.run(
    vza=0., 
    vaa=0.,
    spp=32000
)

Note that since we defined our scene in kilometres, the recorded radiance is in W/km^2/sr/nm.

We start the simulation with the `run` method. We specify the angular grid, as well as the number of samples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

zenith_grid = np.arange(0., 90., 10.)
azimuth_grid = np.arange(0., 360.1, 10.)

result = solver.run(
    vza=zenith_grid, 
    vaa=azimuth_grid,
    spp=4096
)

In [ ]:
r, theta = np.meshgrid(zenith_grid, np.radians(azimuth_grid))
values = np.transpose(result)

fig, ax = plt.subplots(subplot_kw=dict(projection='polar'))
pcm = ax.pcolormesh(theta, r, values, cmap="BuPu_r")
 
plt.colorbar(pcm)
plt.show()
plt.close()

At this point, you can notice the noise in the output: the number of samples has been kept relatively low for this demo. There are lots of possible improvements to this solver and the scene elements we used, and they will be implemented gradually.

This is the end of this demonstration. Eradiate has an even higher level of automation in store, which is showcased in the third part!